# FAQ

## Symptom

```
ValueError: Shapes (None, <num_classes>) and (None, 1) are incompatible
```

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [22]:
inputs = layers.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # --------------------------------------------------------------------------------
    # Causes "ValueError: Shapes (None, 10) and (None, 1) are incompatible"
    # --------------------------------------------------------------------------------
    metrics=[
        [tf.keras.metrics.AUC(from_logits=False, name="auc")]
    ]
    # --------------------------------------------------------------------------------
)

In [23]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/2


ValueError: in user code:

    /home/oonisim/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /home/oonisim/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/oonisim/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/oonisim/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/oonisim/conda/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/oonisim/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/

# Cause

1. ```tf.keras.metrics.AUC``` is **for binary classification** with ```BinaryCrossentropy```.
2. The task is **multi-categorical** with ```SparseCategoricalCrossentropy```

Use ```AUC``` metric for BinaryCrossentropy.

* [Keras Metrics - AUC class](https://keras.io/api/metrics/classification_metrics/#auc-class)

```
# Reports the AUC of a model outputing a probability.
model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])

# Reports the AUC of a model outputing a logit.
model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
```

## Message
It is saying 

* ```tf.keras.metrics.AUC``` is expecting the shape ```(batch_size, 1)``` or ```(None, 1)```. 
* However the input shape to AUC is ```(batch_size, 10)``` as there are 10 digits to classify.

In [ ]:
# Solution

Use 